In [3]:
# task_3_hypothesis_testing/calculate_metrics.py
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
# Load data (assuming DVC-tracked data)
data =  pd.read_csv(r"C:\10x AIMastery\Insurance-risk-analytics\data\insurance_data.csv.dvc") # Update path as needed

In [1]:
# task_3_hypothesis_testing/segmentation.py
from scipy.stats import chi2_contingency


In [6]:
# Create required columns
data['HasClaim'] = data['TotalClaims'] > 0
data['Margin'] = data['TotalPremium'] - data['TotalClaims']

In [8]:
# Segment by Province
province_groups = data.groupby('Province').agg({
    'HasClaim': 'mean',  # Claim Frequency
    'TotalClaims': lambda x: x[x > 0].mean() if x[x > 0].size > 0 else np.nan,  # Claim Severity
    'Margin': 'mean',    # Average Margin
    'Province': 'count'  # Sample size (using Province as proxy for PolicyID)
}).rename(columns={
    'HasClaim': 'ClaimFrequency',
    'TotalClaims': 'ClaimSeverity',
    'Margin': 'Margin',
    'Province': 'SampleSize'
}).reset_index()

print("Province Segmentation:")
print(province_groups)

# Save segmentation results
province_groups.to_csv(r"C:\10x AIMastery\Insurance-risk-analytics\task_3_hypothesis_testing\province_segmentation.csv", index=False)

Province Segmentation:
        Province  ClaimFrequency  ClaimSeverity  Margin  SampleSize
0  KwaZulu-Natal             1.0          300.0   900.0           3


In [9]:
# Check for confounding variables (e.g., Gender distribution across provinces)
gender_province_crosstab = pd.crosstab(data['Province'], data['Gender'])
chi2, p, dof, expected = chi2_contingency(gender_province_crosstab)
print(f"Chi-squared test for Gender distribution across Provinces: p-value = {p:.4f}")
if p < 0.05:
    print("Warning: Gender distribution differs significantly across provinces.")

Chi-squared test for Gender distribution across Provinces: p-value = 1.0000
